# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvfile = 'Weather_DF.csv'
weather_df = pd.read_csv(csvfile)
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Barcelos,41.54,-8.62,51.80,71,75,13.87,PT,1607049940
1,1,San Quintín,30.48,-115.95,57.97,74,0,0.67,MX,1607049940
2,2,Mar del Plata,-38.00,-57.56,57.99,82,90,5.82,AR,1607049765
3,3,Ābu Road,24.48,72.78,73.67,29,0,2.53,IN,1607049940
4,4,Cape Town,-33.93,18.42,62.01,72,0,25.28,ZA,1607049902


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
latlonglocations = weather_df[["Lat", "Lng"]]
weight = weather_df["Humidity"].astype(float)
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '2px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(latlonglocations, weights=weight, 
                                 dissipating=False, max_intensity = 100,
                                 point_radius=2)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

#My weather conditions: Max Temp </= 80, windspeed < 10, < 60 cloudiness, humidity < 40
# I am going to do a .loc with conjunctions to find my desired cities.
myweather_df = weather_df.loc[(weather_df['Max Temp'] <= 80)\
                              & (weather_df['Wind Speed'] <= 10)\
                              & (weather_df['Cloudiness'] < 60)\
                             & (weather_df['Humidity']< 35)]
# Drop null/NA values and display DF
myweather_df.dropna()
myweather_df = myweather_df.drop(columns=['Unnamed: 0'])
myweather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Ābu Road,24.48,72.78,73.67,29,0,2.53,IN,1607049940
15,Saint George,37.10,-113.58,41.00,19,1,3.36,US,1607049852
69,Bassar,9.25,0.78,70.72,24,0,2.33,TG,1607049946
115,Kiryat Gat,31.61,34.76,57.00,10,0,3.85,IL,1607049951
119,Kidal,18.44,1.41,70.11,23,15,4.00,ML,1607049951
122,Aranos,-24.13,19.12,68.50,25,0,0.74,NaN,1607049952
139,San Rafael,-34.62,-68.33,62.85,31,0,2.33,AR,1607049953
158,Sokolo,14.73,-6.12,73.29,21,0,5.84,ML,1607049955
205,Gaza,31.50,34.47,57.00,10,0,5.79,PS,1607049959
334,Gogrial,8.53,28.10,69.42,26,15,3.96,SS,1607049972


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = myweather_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lng']=""
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
3,Ābu Road,IN,24.48,72.78,,,
15,Saint George,US,37.10,-113.58,,,
69,Bassar,TG,9.25,0.78,,,
115,Kiryat Gat,IL,31.61,34.76,,,
119,Kidal,ML,18.44,1.41,,,
122,Aranos,NaN,-24.13,19.12,,,
139,San Rafael,AR,-34.62,-68.33,,,
158,Sokolo,ML,14.73,-6.12,,,
205,Gaza,PS,31.50,34.47,,,
334,Gogrial,SS,8.53,28.10,,,


In [7]:
# Set up Parameter variables
target_search = "hotel"
target_radius = 5000
target_type = "lodging"
# set up a Parameters dictionary
params = {"keyword": target_search,"radius": target_radius,"type": target_type,"key": g_key}
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

In [8]:
print('''
---------------------
Beginning your search
---------------------
''')
for index, row in hotel_df.iterrows():
    # Retrieve hotels but I have to go by the lat and long
    lat = row["Lat"]
    long = row["Lng"]
    name = row["City"]
    # add keyword to params dict
    params["location"] = (f"{lat},{long}")
    # assemble url and make API request
    print(f"Retrieving Results for {name}.")
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    try:
        print(f"The closest hotel to {name} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        hotel_df.loc[index, "Hotel Lat"] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, "Hotel Lng"] = results[0]['geometry']['location']['lng']
        
    except(KeyError, IndexError):
        print(f"Could not find a hotel for {name}... skipping to next vacation spot.")
        hotel_df.loc[index, "Hotel Name"] = "Not Found"
        hotel_df.loc[index, "Hotel Lat"] = hotel_df.loc[index, "Lat"]
        hotel_df.loc[index, "Hotel Lng"] = hotel_df.loc[index, "Lng"]
    print("------------")
    
print('''
------------------------
The search has concluded
------------------------
''')


---------------------
Beginning your search
---------------------

Retrieving Results for Ābu Road.
Could not find a hotel for Ābu Road... skipping to next vacation spot.
------------
Retrieving Results for Saint George.
Could not find a hotel for Saint George... skipping to next vacation spot.
------------
Retrieving Results for Bassar.
Could not find a hotel for Bassar... skipping to next vacation spot.
------------
Retrieving Results for Kiryat Gat.
Could not find a hotel for Kiryat Gat... skipping to next vacation spot.
------------
Retrieving Results for Kidal.
Could not find a hotel for Kidal... skipping to next vacation spot.
------------
Retrieving Results for Aranos.
Could not find a hotel for Aranos... skipping to next vacation spot.
------------
Retrieving Results for San Rafael.
Could not find a hotel for San Rafael... skipping to next vacation spot.
------------
Retrieving Results for Sokolo.
Could not find a hotel for Sokolo... skipping to next vacation spot.
-----------

In [9]:
hotel_df['Hotel Lat'] = hotel_df['Hotel Lat'].astype(float)
hotel_df['Hotel Lng'] = hotel_df['Hotel Lng'].astype(float)
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
3,Ābu Road,IN,24.48,72.78,Not Found,24.48,72.78
15,Saint George,US,37.10,-113.58,Not Found,37.10,-113.58
69,Bassar,TG,9.25,0.78,Not Found,9.25,0.78
115,Kiryat Gat,IL,31.61,34.76,Not Found,31.61,34.76
119,Kidal,ML,18.44,1.41,Not Found,18.44,1.41
122,Aranos,NaN,-24.13,19.12,Not Found,-24.13,19.12
139,San Rafael,AR,-34.62,-68.33,Not Found,-34.62,-68.33
158,Sokolo,ML,14.73,-6.12,Not Found,14.73,-6.12
205,Gaza,PS,31.50,34.47,Not Found,31.50,34.47
334,Gogrial,SS,8.53,28.10,Not Found,8.53,28.10


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


# Display figure


In [12]:
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '2px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
gmaps.figure(layout=figure_layout)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…